# RC4 Cypher Method

## First of all, let's define the RC4 cypher method.

In [31]:
def RC4(message, key):
    K = key# "thisisaverylongkey"
    T = []
    S = []

    ciphered_message = []

    for i in range(256):
        S.append(i)
        T.append(K[i % len(K)])

    j = 0
    for i in range(256):
        j = (j + S[i] + ord(T[i])) % 256
        S[i], S[j] = S[j], S[i]

    i, j = 0, 0

    for l in range(len(message)):
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        if type(message) == str:
            ciphered_message.append(chr(ord(message[l]) ^ S[(S[i] + S[j]) % 256]))
        elif type(message) == list:
            ciphered_message.append(chr(ord(message[l]) ^ S[(S[i] + S[j]) % 256]))

    
    # print(ciphered_message)
    return ciphered_message


The code for work needs a key. The key is a sequence of bytes or chars that we use to encrypt or decrypt the data. This method works as cypher and a decypher using the same key.

For example, we use the key defined as key on the code as "thisisaverylongkey". So, lets cypher and decypher the message "this is a very long message".

In [32]:
message = "this is a very long message"
key = "thisisaverylongkey"
ciphered_message = RC4(message, key)
print("cyphered text" + str(ciphered_message))
print("decyphered text" + str(RC4(ciphered_message, key)))

cyphered text['¯', '\xa0', 'ð', '\x12', 'Ü', 'Á', '\x9c', 'C', 'Ð', 'Þ', 'a', '\x07', '\n', '#', 'ú', '\x11', 'P', '§', '9', 'ç', 'î', 'Ó', '\xad', '\x00', '¼', 'Æ', 'Ø']
decyphered text['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', ' ', 'v', 'e', 'r', 'y', ' ', 'l', 'o', 'n', 'g', ' ', 'm', 'e', 's', 's', 'a', 'g', 'e']


We can rewrite the code encapsulating each phase of the cypher. In this case we want to encapsulate the intialization phase:

In [40]:
def RC4_initialization(key):
    K = key # "thisisaverylongkey"
    T = []
    S = []

    for i in range(256):
        S.append(i)
        T.append(K[i % len(K)])

    j = 0
    for i in range(256):
        j = (j + S[i] + ord(T[i])) % 256
        S[i], S[j] = S[j], S[i]

    return T, S, K

def RC4_encapsulated(message, key):
    T, S, K = RC4_initialization(key)
    ciphered_message = []
    i, j = 0, 0

    for l in range(len(message)):
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        if type(message) == str:
            ciphered_message.append(chr(ord(message[l]) ^ S[(S[i] + S[j]) % 256]))
        elif type(message) == list:
            ciphered_message.append(chr(ord(message[l]) ^ S[(S[i] + S[j]) % 256]))

    
    # print(ciphered_message)
    return ciphered_message

In [41]:
message = "this is a very long message"
key = "thisisaverylongkey"
ciphered_message = RC4_encapsulated(message, key)
print("cyphered text" + str(ciphered_message))
print("decyphered text" + str(RC4_encapsulated(ciphered_message, key)))

cyphered text['¯', '\xa0', 'ð', '\x12', 'Ü', 'Á', '\x9c', 'C', 'Ð', 'Þ', 'a', '\x07', '\n', '#', 'ú', '\x11', 'P', '§', '9', 'ç', 'î', 'Ó', '\xad', '\x00', '¼', 'Æ', 'Ø']
decyphered text['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 'a', ' ', 'v', 'e', 'r', 'y', ' ', 'l', 'o', 'n', 'g', ' ', 'm', 'e', 's', 's', 'a', 'g', 'e']


Now, lets prove it

## Now, lets calculate an array T to mantain array S intact after initialization. 

As K is needed to calculate T, so, lets find a key K to calculate T that achieves our wanted result. So, we don't actually need to know the key, because we are going to calculate the key.

Lets analyse the code below. corresponding to initailization phase.

```python
# As we now are going to calculate the key, let's comment it
# K = "thisisaverylongkey"
T = []
S = []

# message = ['¯', '\xa0', 'ð', '\x12', 'Ü', 'Á', '\x9c', 'C', 'Ð', 'Þ', 'a', '\x07', '\n', '#', 'ú', '\x11', 'P', '§', '9', 'ç', 'î', 'Ó', '\xad', '\x00', '¼', 'Æ', 'Ø']
# message = "this is a very long message"
ciphered_message = []

for i in range(256):
    S.append(i)
    # T.append(K[i % len(K)]) We also are going to calculate T array, so, let's comment this line too

j = 0
for i in range(256):
    # In this code line we see that to mantain S intact, we need j to be equal to the value of i in that way, 
    # when we swap variables S[i], S[j] = S[j], S[i] == S[i], S[i] = S[i], S[i].
    # For it to happen we need to j + T[i] == 256, to be cancelled with the modulus 256, and as S[i] got the value of i, j will be equal to i.
    # So, our array T must be equal to a multiple of 256 - i for each T[i] item.
    j = (j + S[i] + ord(T[i])) % 256 
    S[i], S[j] = S[j], S[i]
```

Lets code it!

In [64]:
# Let be n a positive integer to get a multiple of 256 - i on each item of the array T.

def arrayT_gen(n):
    if n == 0:
        n = 1
    elif n < 0:
        n = -n
    T = []
    for i in range(256):
        if i > 0:
            T.append(((256 * n) - i) + 1)
        else:
            T.append(((256 * n) - i))
    return T

def key_gen(n):
    T = arrayT_gen(n)
    k = []
    for i in range(len(T)):
        k.append(chr(T[i]))
    return k

Now, lets generate a key K to calculate T.

In [65]:
k = key_gen(1)
print(k)

['Ā', 'Ā', 'ÿ', 'þ', 'ý', 'ü', 'û', 'ú', 'ù', 'ø', '÷', 'ö', 'õ', 'ô', 'ó', 'ò', 'ñ', 'ð', 'ï', 'î', 'í', 'ì', 'ë', 'ê', 'é', 'è', 'ç', 'æ', 'å', 'ä', 'ã', 'â', 'á', 'à', 'ß', 'Þ', 'Ý', 'Ü', 'Û', 'Ú', 'Ù', 'Ø', '×', 'Ö', 'Õ', 'Ô', 'Ó', 'Ò', 'Ñ', 'Ð', 'Ï', 'Î', 'Í', 'Ì', 'Ë', 'Ê', 'É', 'È', 'Ç', 'Æ', 'Å', 'Ä', 'Ã', 'Â', 'Á', 'À', '¿', '¾', '½', '¼', '»', 'º', '¹', '¸', '·', '¶', 'µ', '´', '³', '²', '±', '°', '¯', '®', '\xad', '¬', '«', 'ª', '©', '¨', '§', '¦', '¥', '¤', '£', '¢', '¡', '\xa0', '\x9f', '\x9e', '\x9d', '\x9c', '\x9b', '\x9a', '\x99', '\x98', '\x97', '\x96', '\x95', '\x94', '\x93', '\x92', '\x91', '\x90', '\x8f', '\x8e', '\x8d', '\x8c', '\x8b', '\x8a', '\x89', '\x88', '\x87', '\x86', '\x85', '\x84', '\x83', '\x82', '\x81', '\x80', '\x7f', '~', '}', '|', '{', 'z', 'y', 'x', 'w', 'v', 'u', 't', 's', 'r', 'q', 'p', 'o', 'n', 'm', 'l', 'k', 'j', 'i', 'h', 'g', 'f', 'e', 'd', 'c', 'b', 'a', '`', '_', '^', ']', '\\', '[', 'Z', 'Y', 'X', 'W', 'V', 'U', 'T', 'S', 'R', 'Q', 'P', 'O'

Now, lets put it together on our initialization phase and see our arrays:

In [69]:
key = key_gen(1)

T, S, k = RC4_initialization(key)

print(S)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

As we see, S before initialization is equal to S after initialization. Both are arrays from 0 to 255.